In [ ]:
from kafka3 import KafkaProducer
from datetime import datetime
from time import sleep
from json import dumps
import datetime as dt
import random
import csv

In [ ]:
host_ip = "118.139.86.28"

In [ ]:
#reading the csv file
with open ("hotspot_AQUA_streaming.csv", "r") as file:
    AQUA_file = file.readlines()
    AQUA_file = [line.strip() for line in AQUA_file]
    for i in range(len(AQUA_file)):
        AQUA_file[i] = AQUA_file[i].split(",")

In [ ]:
#Create a Kafka producer
def connect_kafka_producer():
    _producer = None
    try: 
        _producer = KafkaProducer(bootstrap_servers=[f'{host_ip}:9092'],
                                  value_serializer=lambda x:dumps(x).encode('ascii'),
                                  api_version=(0, 10),
                                 client_id = ("hotspotAQUA"))
        
    except Exception as ex:
        print("Exception while connecting Kafka")
        print(str(ex))
        
    finally:
        return _producer

#publish messages
def publish_message(producer_instance, topic_name, key, data):
    try:
        key_bytes = bytes(key, encoding = 'utf-8')
        
        producer_instance.send(topic_name, key=key_bytes, value=data)
        producer_instance.flush()
        
        print("Message successfully publish. Data: " + str(data))
        
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))

In [ ]:
def addDateTime(currDateTime):
    currDateTime = datetime.strptime(currDateTime, '%Y-%m-%dT%H:%M:%S')
    newDateTime = currDateTime + dt.timedelta(hours = 4, minutes=48)
    
    return newDateTime.isoformat()

In [ ]:
topic = "hotspotTopic"
# topic = "fires"

print("Publishing records...")
producer = connect_kafka_producer()
prod_id = producer.config["client_id"]

newDateTime = '2023-01-02T00:00:00'

while True:
    randIndex = random.randint(1,len(AQUA_file)-1)
    newDateTime = addDateTime(newDateTime)
    row = AQUA_file[randIndex]
    
    hotspotAQUA_record = {
        "producer_info": prod_id,
        "datetime": newDateTime,
        "latitude": float(row[0]),
        "longitude": float(row[1]),
        "confidence": int(row[2]),
        "surface_temperature": int(row[3])
    }
    publish_message(producer, topic, 'hotspotAQUA', hotspotAQUA_record)
    sleep(2) #produce every 2 seconds 